In [1]:
# import from script
from diedfunctions import * # 'Kernel restart & clear output' to load new functions
import os

In [2]:
lemmatize?

In [ ]:
# textfile = f'../../../enwiki_city_pairs/paris_milan.txt'

# with open(textfile, 'r', encoding='utf-16') as f:
#     parismilan = [x.strip() for x in f.read().split('\n') if len(x) and 'title=' not in x]
    
# parismilan[:10]

In [43]:
directory = f'../../../enwiki_city_pairs/'
city_pairs_text = []

for file in os.listdir(directory):
    if file.endswith('.txt'):
        with open(directory+file, encoding='utf-16') as f:
            city_pair_text = ' '.join([x.strip() for x in f.read().split('\n') if len(x) and 'title' not in x])
            city_pairs_text.append(city_pair_text)

In [2]:
# city_pairs_text[7]

In [5]:
texts = city_pairs_text
nlp.max_length = 1500000
processed_text = [text for text in tqdm(nlp.pipe(texts, n_process=-1, disable=["ner", "parser"]), total=len(texts))]
lemmatized_text = [[word.lemma_.lower() for word in text if word.pos_ in POSfilter and not word.is_punct and not word.is_stop] for text in processed_text]
regexed_text = [[re.sub(r'\W+', '', word) for word in text] for text in lemmatized_text]

  0%|          | 0/10 [00:00<?, ?it/s]

NameError: name 'POSfilter' is not defined

In [6]:
lemmatized_texts = lemmatize(city_pairs_text)

  0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
dictionary, corpus = vectorize(lemmatized_texts)

In [20]:
from gensim.models.wrappers import LdaMallet
from gensim.models import CoherenceModel
PATH_TO_MALLET = r'C:/mallet/bin/mallet.bat'

def train_model(lemmatized_text, dictionary=[], corpus=[], MIN_DF = 1, MAX_DF = 0.6, N_TOPICS = 10, N_ITERATIONS = 1000):
    # usually 1000 iterations will do
    
#     # Call vectorization function if either dictionary or corpus is missing as parameter
#     if not type(dictionary) == gensim.corpora.dictionary.Dictionary or not corpus:
#         dictionary, corpus = vectorize(lemmatized_text, MIN_DF, MAX_DF)
    
    
    lda_model = LdaMallet(PATH_TO_MALLET,
                corpus=corpus,
                id2word=dictionary,
                num_topics=N_TOPICS,
                optimize_interval=10,
                iterations=N_ITERATIONS)
    
    coherence_score = CoherenceModel(model=lda_model, texts=lemmatized_text, dictionary=dictionary, coherence='c_v').get_coherence()
    
    return(lda_model, coherence_score, dictionary, corpus)

In [47]:
lda_model, coherence_score, dictionary, corpus = train_model(lemmatized_texts, dictionary, corpus, N_TOPICS=7)

In [46]:
coherence_score

0.5971974996342317

In [38]:
MAX_WORDS = 20
N_TOPICS = 10

for i in range(N_TOPICS):
    words = lda_model.show_topic(i, topn=MAX_WORDS)
    print([(x[0], round(x[1], 3)) for x in words])

[('henderson', 0.005), ('zoo', 0.005), ('chamberlain', 0.004), ('ribbentrop', 0.004), ('freud', 0.003), ('bismarck', 0.003), ('rabbi', 0.003), ('speciman', 0.003), ('tuberculosis', 0.003), ('trilogy', 0.002), ('decker', 0.002), ('specimen', 0.002), ('bird', 0.002), ('saatchi', 0.002), ('feather', 0.002), ('anthology', 0.002), ('ogilvie', 0.002), ('sequence', 0.002), ('hirsch', 0.002), ('hook', 0.002)]
[('uefa', 0.01), ('silva', 0.006), ('card', 0.006), ('juventus', 0.005), ('atlético', 0.005), ('audi', 0.005), ('ac', 0.004), ('injury', 0.004), ('liga', 0.004), ('beckham', 0.004), ('penalty', 0.003), ('fc', 0.003), ('pérez', 0.003), ('semi', 0.003), ('baggio', 0.003), ('populous', 0.003), ('chelsea', 0.003), ('fee', 0.003), ('bennati', 0.003), ('tapié', 0.003)]
[('mosque', 0.006), ('emil', 0.004), ('carol', 0.004), ('adorno', 0.003), ('astrup', 0.003), ('abel', 0.003), ('inönü', 0.003), ('tsvetaeva', 0.003), ('mayr', 0.002), ('bootie', 0.002), ('munch', 0.002), ('kraus', 0.002), ('japan

In [28]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import gensim

pyLDAvis.enable_notebook()

# dictionary, corpus = vectorize(lemmatized_text, MIN_DF=1, MAX_DF=0.6)
# model, coherence, dictionary, corpus = train_model(lemmatized_text=lemmatized_text, dictionary=[], corpus=[], N_TOPICS=4)

lda_conv = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_model)
gensimvis.prepare(lda_conv, corpus, dictionary)


c:\users\deejay\personal files [local]\applied data science\thesis - citynet\citynet\venv_citynet\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
c:\users\deejay\personal files [local]\applied data science\thesis - citynet\citynet\venv_citynet\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
c:\users\deejay\personal files [local]\applied data science\thesis - citynet\citynet\venv_citynet\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.047990  0.009179       1        1  43.050976
6     -0.129449 -0.059401       2        1  10.479277
4     -0.176163  0.049094       3        1   8.959610
3      0.026101 -0.255158       4        1   7.844764
7      0.056885 -0.260794       5        1   6.416101
8      0.035612  0.008024       6        1   5.753943
5     -0.019791  0.159830       7        1   5.375565
1     -0.241326  0.114213       8        1   4.177773
2      0.271951  0.013649       9        1   3.977324
0      0.224170  0.221363      10        1   3.964668, topic_info=            Term        Freq       Total Category  logprob  loglift
5078         nyu  116.000000  116.000000  Default  30.0000  30.0000
3561   henderson   85.000000   85.000000  Default  29.0000  29.0000
8589        uefa   38.000000   38.000000  Default  28.0000  28.0000
7391         zoo   42.000000   42.000000  Default  27.0000  27.0000
2882    eurostar   46.000000   46.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
2952       faith    5.815588   11.911588  Topic10  -6.2922   2.5108
1356  background    5.815588   13.036867  Topic10  -6.2922   2.4205
3102      forbes    5.815588   18.227643  Topic10  -6.2922   2.0854
164       cooper    4.846324   11.893665  Topic10  -6.4745   2.3300
305        fritz    4.846324   10.875902  Topic10  -6.4745   2.4194

[895 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
13623      4  1.009219       0
7425       3  0.985031      01
13624      5  1.013305     10b
856       10  1.031710     143
13626      4  1.009219   150th
...      ...       ...     ...
7400       7  0.865561      zu
7400      10  0.123652      zu
10342      7  0.359811    özil
10342      8  0.629670    özil
21719      4  1.009219  ʻabdul

[1369 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 7, 5, 4, 8, 9, 6, 2, 3, 1])

In [39]:
import pandas as pd

transformed_docs = lda_model.load_document_topics()
topic_distributions = pd.DataFrame([[x[1] for x in doc] for doc in transformed_docs], 
             columns=['topic_{}'.format(i) for i in range(10)])
topic_distributions.head(12)

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,0.000012,0.000162,0.000012,0.000012,0.000053,0.001071,0.001936,0.000012,0.967271,0.029458
1,0.262851,0.000176,0.000001,0.000001,0.000005,0.066575,0.000005,0.000001,0.090656,0.579728
2,0.000005,0.098607,0.000005,0.000005,0.648723,0.129791,0.000022,0.000005,0.000055,0.122783
3,0.000004,0.018955,0.000004,0.000004,0.000016,0.000040,0.636509,0.000004,0.246105,0.098359
4,0.000011,0.017629,0.000011,0.000011,0.000047,0.916011,0.000052,0.000011,0.000130,0.066089
5,0.000007,0.999539,0.000007,0.000007,0.000031,0.000077,0.000034,0.000007,0.000085,0.000205
6,0.000001,0.000015,0.279945,0.000001,0.000005,0.103163,0.000005,0.000001,0.052851,0.564012
7,0.007313,0.173703,0.085870,0.000336,0.091739,0.142644,0.011907,0.486488,NaN,NaN
8,0.000004,0.076690,0.000004,0.000004,0.581304,0.198687,0.000017,0.000004,0.000044,0.143244
9,0.000003,0.133675,0.000003,0.000003,0.000013,0.000033,0.627357,0.000003,0.125126,0.113784


In [35]:
topic_distributions.mean().sort_values(ascending=False)

topic_9    0.190851
topic_8    0.164703
topic_5    0.155809
topic_1    0.151915
topic_4    0.132194
topic_6    0.127785
topic_7    0.048654
topic_2    0.036586
topic_0    0.027021
topic_3    0.000038
dtype: float64

In [40]:
topic_distributions.idxmax(axis=1)

0    topic_8
1    topic_9
2    topic_4
3    topic_6
4    topic_5
5    topic_1
6    topic_9
7    topic_7
8    topic_4
9    topic_6
dtype: object

In [53]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(PATH_TO_MALLET, corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=lemmatized_texts, start=2, limit=30, step=3)